In [1]:
import pandas as pd

In [4]:

# Путь к файлу
file_path = "data/med_production_data_dirty.csv"

# Попробуем считать файл с указанием кодировки, которая может содержать такие ошибки — 'utf-8-sig' или 'cp1251'
# Начнем с чтения без указания, затем попробуем перекодировку
try:
    df = pd.read_csv(file_path)
except UnicodeDecodeError:
    df = pd.read_csv(file_path, encoding='cp1251')

df.to_csv(file_path, index=False, encoding='cp1251')

In [3]:
df

,BatchID,Date,Completed,InputMass_kg,SmeltTemp_C,SmeltDuration_h,RefineDuration_h,RefineEnergy_kWh,CoolingTime_h,HeatTreatTemp_C,HeatTreatDuration_h,FinalMass_kg,Purity_pct,Quality
0,Batch_0001,2025-11-24,1,8753.57,1068.3,4.0,18.7,3732.4,4.1,NaN,NaN,8567.96,98.95,Низкий
1,Batch_0002,2025-01-13,1,4917.02,1089.8,5.3,23.2,2582.5,2.4,373.0,1.9,4776.43,98.78,низкий
2,Batch_0003,2025-05-06,1,6962.07,1067.7,3.7,29.4,3931.3,2.9,NaN,NaN,6734.89,99.12,Стандарт
3,Batch_0004,2025-03-13,1,6475.88,1090.6,6.7,21.1,3325.0,3.6,337.0,2.9,6288.11,99.82,Стандарт
4,Batch_0005,2025-02-22,1,6989.50,1083.3,4.9,21.3,3657.5,2.8,328.0,2.6,6590.97,99.88,Стандарт
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Batch_0383,2025-08-19,1,7334.64,1090.4,6.8,26.6,3860.3,2.8,NaN,NaN,7107.58,98.67,Низкий
996,Batch_0531,2025-04-08,1,4914.61,1096.2,5.0,25.5,2148.9,2.5,427.0,2.1,4764.87,98.97,Низкий
997,Batch_0432,2025-10-21,1,7516.39,1079.9,3.7,27.0,3701.7,3.9,492.0,1.1,7239.72,98.73,низкий
998,Batch_0911,2025-08-12,1,8491.15,1082.6,6.9,26.0,3557.1,3.6,419.0,1.6,8172.26,99.25,Стандарт


In [15]:
import pandas as pd

# Загрузка файла
df = pd.read_csv("data/med_production_data_dirty.csv", encoding='cp1251')

# Фильтруем только строки, где Completed == 1
df_completed = df[df["Completed"] == 1]

# Считаем строки, где есть хотя бы один пропуск (NaN) в других столбцах
incomplete_rows = df_completed[df_completed.isnull().any(axis=1)]

# Вывод количества таких строк
print(f"Строк с Completed = 1 и неполными данными: {len(incomplete_rows)}")

Строк с Completed = 1 и неполными данными: 124


шаг 1: удалить дубликаты
ё

In [16]:
df.drop_duplicates(inplace=True)
len(df)

950

шаг 2: удалить строки, где где completed = 1, но данные неполные


In [18]:
mask = (df["Completed"] == 1) & (df.isnull().any(axis=1))
df = df[~mask]

len(df)

833

шаг 3: удалить строки, где final mass > input mass

In [19]:
invalid_mass = df["FinalMass_kg"] > df["InputMass_kg"]

# Фильтрация: оставляем только корректные строки
df = df[~invalid_mass]

len(df)

833

шаг 4: привести все в столбце куолити в один формат

In [20]:
df["Quality"] = df["Quality"].astype(str).str.strip().str.title()

щаг 5: проверка на выбросы хз

In [25]:
numeric_cols = df.select_dtypes(include="number").columns

# Словарь для хранения аномалий
anomalies = {}

for col in numeric_cols:
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    mask = (df[col] < lower_bound) | (df[col] > upper_bound)
    anomalies[col] = df[mask]

# Посмотреть, где есть выбросы
for col, rows in anomalies.items():
    print(f"{col}: {len(rows)} выбросов")

Completed: 99 выбросов
InputMass_kg: 0 выбросов
SmeltTemp_C: 17 выбросов
SmeltDuration_h: 0 выбросов
RefineDuration_h: 0 выбросов
RefineEnergy_kWh: 0 выбросов
CoolingTime_h: 0 выбросов
HeatTreatTemp_C: 0 выбросов
HeatTreatDuration_h: 0 выбросов
FinalMass_kg: 99 выбросов
Purity_pct: 8 выбросов


In [22]:
for col in numeric_cols:
    if (df[col] < 0).any():
        print(f"{col} содержит отрицательные значения")

In [29]:
columns_with_outliers = ["SmeltTemp_C"]

# Словарь для хранения масок выбросов
outlier_masks = {}

for col in columns_with_outliers:
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    outlier_masks[col] = (df[col] < lower) | (df[col] > upper)

# Объединяем маски (логическое ИЛИ)
combined_mask = outlier_masks["SmeltTemp_C"]

# Выводим строки с выбросами в этих колонках
df_outliers = df[combined_mask]

# Посмотреть — можно в виде таблицы
print(df_outliers[["BatchID","Completed", "SmeltTemp_C"]])

        BatchID  Completed  SmeltTemp_C
5    Batch_0006          1       1963.1
114  Batch_0115          1       1041.6
137  Batch_0138          1       1883.2
236  Batch_0237          1       1993.0
248  Batch_0249          1       1833.1
255  Batch_0256          1       1960.9
379  Batch_0380          1       1807.9
467  Batch_0468          1       1131.2
477  Batch_0478          1       1130.2
479  Batch_0480          1       1903.6
499  Batch_0500          1       1129.0
578  Batch_0579          1       1949.6
613  Batch_0614          1       1911.6
692  Batch_0693          1       1129.1
722  Batch_0723          1       1890.2
822  Batch_0823          1       1854.6
840  Batch_0841          1       1975.9


In [30]:
def correct_temperature(temp):
    if temp > 1500:
        return (temp - 32) * 5 / 9  # convert from °F to °C
    return temp

# Применим к колонке
df["SmeltTemp_C"] = df["SmeltTemp_C"].apply(correct_temperature)


In [31]:
numeric_cols = df.select_dtypes(include="number").columns

# Словарь для хранения аномалий
anomalies = {}

for col in numeric_cols:
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    mask = (df[col] < lower_bound) | (df[col] > upper_bound)
    anomalies[col] = df[mask]

# Посмотреть, где есть выбросы
for col, rows in anomalies.items():
    print(f"{col}: {len(rows)} выбросов")

Completed: 99 выбросов
InputMass_kg: 0 выбросов
SmeltTemp_C: 14 выбросов
SmeltDuration_h: 0 выбросов
RefineDuration_h: 0 выбросов
RefineEnergy_kWh: 0 выбросов
CoolingTime_h: 0 выбросов
HeatTreatTemp_C: 0 выбросов
HeatTreatDuration_h: 0 выбросов
FinalMass_kg: 99 выбросов
Purity_pct: 8 выбросов


In [32]:
columns_with_outliers = ["SmeltTemp_C"]

# Словарь для хранения масок выбросов
outlier_masks = {}

for col in columns_with_outliers:
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    outlier_masks[col] = (df[col] < lower) | (df[col] > upper)

# Объединяем маски (логическое ИЛИ)
combined_mask = outlier_masks["SmeltTemp_C"]

# Выводим строки с выбросами в этих колонках
df_outliers = df[combined_mask]

# Посмотреть — можно в виде таблицы
print(df_outliers[["BatchID","Completed", "SmeltTemp_C"]])

        BatchID  Completed  SmeltTemp_C
114  Batch_0115          1  1041.600000
137  Batch_0138          1  1028.444444
248  Batch_0249          1  1000.611111
318  Batch_0319          1  1126.500000
379  Batch_0380          1   986.611111
467  Batch_0468          1  1131.200000
477  Batch_0478          1  1130.200000
479  Batch_0480          1  1039.777778
499  Batch_0500          1  1129.000000
613  Batch_0614          1  1044.222222
692  Batch_0693          1  1129.100000
722  Batch_0723          1  1032.333333
743  Batch_0744          1  1125.700000
822  Batch_0823          1  1012.555556
